# Utilities

> Numpy-based cosine similarity optimized to generate same- and different-source scores

In [ ]:
# | default_exp utils

In [ ]:
# | export
import numpy as np

In [ ]:
# | export


def cosine_similarity(X, Z):
    # Normalize the embeddings
    X = X / np.linalg.norm(X, axis=1, keepdims=True)
    Z = Z / np.linalg.norm(Z, axis=1, keepdims=True)

    # Compute the dot product between the normalized embeddings
    dot_product = np.dot(X, Z.T)

    # Return the cosine similarity between the embeddings
    return dot_product

In [ ]:
# | export


def compute_ss_ds(X, x_id, x_names=None, Z=None, z_id=None, z_names=None):
    """
    Compute cosine similarities between the cartesian product of two arrays X and Z and
    return same-source (ss) and different-source (ds) scores.
    If only the array X and x_id are provided, compute the cosine similarities between all pairwise
    combination in X. Also return the names of the files associated with each score, is x_names and z_names are provided.

    Inputs:
        X, Z: 2d numpy arrays with embeddings (1 per line)
        x_id, z_id: 1d numpy arrays with identity labels
        x_names, z_names: 1d numpy arrays with names of files associated with the embeddings

    Returns:
        scores: 1d numpy array with scores
        y: 1d numpy arrays with ss (1) and ds (0) labels to the scores array
        names: list of tuples with names of files associated with each score
    """
    assert X.ndim == 2
    assert X.shape[0] == len(x_id)
    ss_names = None
    ds_names = None
    if Z is None:  # compute scores of X vs X
        similarities = cosine_similarity(X, X)
        ss_mask = x_id[:, np.newaxis] == x_id
        upper_triangle_mask = np.triu(np.ones_like(similarities), k=1).astype(bool)
        ss = similarities[(ss_mask & upper_triangle_mask)]
        ds = similarities[(~ss_mask & upper_triangle_mask)]
        if x_names is not None:  # compute names of X vs X
            assert X.shape[0] == len(x_id) == len(x_names)
            ss_names = [
                (x_names[i], x_names[j])
                for i, j in np.argwhere(upper_triangle_mask)
                if x_id[i] == x_id[j]
            ]
            ds_names = [
                (x_names[i], x_names[j])
                for i, j in np.argwhere(upper_triangle_mask)
                if x_id[i] != x_id[j]
            ]
    if Z is not None:  # compute scores of X vs Z
        assert Z.ndim == 2
        assert Z.shape[0] == len(z_id)
        similarities = cosine_similarity(X, Z)
        ss_mask = x_id[:, np.newaxis] == z_id
        ss = similarities[ss_mask]
        ds = similarities[~ss_mask]
        if z_names is not None:  # compute names of X vs Z
            assert Z.shape[0] == len(z_id) == len(z_names)
            ss_names = [(x_names[i], z_names[j]) for i, j in np.argwhere(ss_mask)]
            ds_names = [(x_names[i], z_names[j]) for i, j in np.argwhere(~ss_mask)]

    scores = np.concatenate([ss, ds])
    y = np.concatenate([np.ones(len(ss)), np.zeros(len(ds))])
    names = (
        ss_names + ds_names if ss_names is not None and ds_names is not None else None
    )
    return scores, y, names

In [ ]:
# Example usage
X = np.random.rand(4, 128)  # 4 embeddings of dimension 128
x_id = np.array([0, 0, 1, 1])  # Identity labels
x_names = np.array(["0_a.jpg", "0_b.jpg", "1_a.jpg", "1_b.jpg"])

Z = np.random.rand(4, 128)  # 4 embeddings of dimension 128
z_id = np.array([0, 1, 1, 2])  # Identity labels
z_names = np.array(["0_c.jpg", "1_c.jpg", "1_d.jpg", "2_a.jpg"])

scores, y, names = compute_ss_ds(
    X=X, x_id=x_id, x_names=x_names, Z=Z, z_id=z_id, z_names=z_names
)
scores, y, names

(array([0.7261676 , 0.76134557, 0.78772832, 0.76922003, 0.74130075,
        0.75798372, 0.77273163, 0.75997033, 0.76467846, 0.71801777,
        0.75326126, 0.73211696, 0.72354578, 0.78339759, 0.74410164,
        0.74325012]),
 array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 [('0_a.jpg', '0_c.jpg'),
  ('0_b.jpg', '0_c.jpg'),
  ('1_a.jpg', '1_c.jpg'),
  ('1_a.jpg', '1_d.jpg'),
  ('1_b.jpg', '1_c.jpg'),
  ('1_b.jpg', '1_d.jpg'),
  ('0_a.jpg', '1_c.jpg'),
  ('0_a.jpg', '1_d.jpg'),
  ('0_a.jpg', '2_a.jpg'),
  ('0_b.jpg', '1_c.jpg'),
  ('0_b.jpg', '1_d.jpg'),
  ('0_b.jpg', '2_a.jpg'),
  ('1_a.jpg', '0_c.jpg'),
  ('1_a.jpg', '2_a.jpg'),
  ('1_b.jpg', '0_c.jpg'),
  ('1_b.jpg', '2_a.jpg')])

In [ ]:
scores, y, names = compute_ss_ds(X=X, x_id=x_id, Z=Z, z_id=z_id)
scores, y, names

(array([0.7261676 , 0.76134557, 0.78772832, 0.76922003, 0.74130075,
        0.75798372, 0.77273163, 0.75997033, 0.76467846, 0.71801777,
        0.75326126, 0.73211696, 0.72354578, 0.78339759, 0.74410164,
        0.74325012]),
 array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 None)

In [ ]:
scores, y, names = compute_ss_ds(X=X, x_id=x_id, x_names=x_names)
scores, y, names

(array([0.75920622, 0.77365152, 0.76394119, 0.78708138, 0.75681996,
        0.79128601]),
 array([1., 1., 0., 0., 0., 0.]),
 [('0_a.jpg', '0_b.jpg'),
  ('1_a.jpg', '1_b.jpg'),
  ('0_a.jpg', '1_a.jpg'),
  ('0_a.jpg', '1_b.jpg'),
  ('0_b.jpg', '1_a.jpg'),
  ('0_b.jpg', '1_b.jpg')])

In [ ]:
scores, y, names = compute_ss_ds(X=X, x_id=x_id)
scores, y, names

(array([0.75920622, 0.77365152, 0.76394119, 0.78708138, 0.75681996,
        0.79128601]),
 array([1., 1., 0., 0., 0., 0.]),
 None)